<a href="https://colab.research.google.com/github/poo5zan/realized-volatility/blob/main/from-kaggle-optiver-challenge.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>


code copied from kaggle notebook, and made changes on top of it  
https://www.kaggle.com/competitions/optiver-realized-volatility-prediction/discussion/274970

In [74]:
# connect to google drive
# from google.colab import drive
# drive.mount('/content/drive')

In [5]:
# import pandas as pd
# p = pd.read_csv("./data/order_trade_agg.csv")

# 3 dimensions
# 1 time 
# 1 stock 
# 1 features 
# 1 batch_size 
# total of 4 dimension 


# input of 4 d 
# output volatility for all stocks 
# output of 3 dimensions : volatility, stocks, batch_size. 


In [2]:
p = pd.read_csv("./data/order_trade_agg.csv")
p.columns

Index(['stock_id', 'time_id', 'target', 'book.seconds_in_bucket.count',
       'book.wap1.sum', 'book.wap1.mean', 'book.wap1.std', 'book.wap2.sum',
       'book.wap2.mean', 'book.wap2.std', 'book.log_return1.sum',
       'book.log_return1.realized_volatility', 'book.log_return1.mean',
       'book.log_return1.std', 'book.log_return2.sum',
       'book.log_return2.realized_volatility', 'book.log_return2.mean',
       'book.log_return2.std', 'book.log_return_ask1.sum',
       'book.log_return_ask1.realized_volatility', 'book.log_return_ask1.mean',
       'book.log_return_ask1.std', 'book.log_return_ask2.sum',
       'book.log_return_ask2.realized_volatility', 'book.log_return_ask2.mean',
       'book.log_return_ask2.std', 'book.log_return_bid1.sum',
       'book.log_return_bid1.realized_volatility', 'book.log_return_bid1.mean',
       'book.log_return_bid1.std', 'book.log_return_bid2.sum',
       'book.log_return_bid2.realized_volatility', 'book.log_return_bid2.mean',
       'book.log_re

In [75]:
# install these if running from google colab
# !pip install darts
# !pip install pytorch_tabnet

In [6]:
# Load Libraries
PYTORCH_ENABLE_MPS_FALLBACK=1
import warnings
warnings.filterwarnings('ignore')

import gc
import glob
import os
import time
import traceback
from contextlib import contextmanager
from enum import Enum
from typing import Dict, List, Optional, Tuple
import seaborn as sns
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import lightgbm as lgb
from IPython.display import display
from joblib import delayed, Parallel
from sklearn.decomposition import LatentDirichletAllocation
from sklearn.manifold import TSNE
from sklearn.model_selection import GroupKFold
from sklearn.neighbors import NearestNeighbors
from sklearn.preprocessing import minmax_scale
# from tqdm import tqdm_notebook as tqdm
from sklearn.metrics import mean_squared_error, mean_absolute_error, mean_absolute_percentage_error
from datetime import datetime
from darts import TimeSeries
from darts.models import TCNModel, RNNModel, TransformerModel
from darts.dataprocessing.transformers import Scaler
from darts.utils.timeseries_generation import datetime_attribute_timeseries
from darts.metrics import mape, r2_score
from darts.utils.missing_values import fill_missing_values
from darts.datasets import AirPassengersDataset, SunspotsDataset, EnergyDataset
from darts.metrics import mae, rmse, mse, mape
import random
from typing import List, Tuple, Optional, Union
import torch
import torch.nn as nn
from sklearn.preprocessing import StandardScaler
from torch.utils.data import Dataset, DataLoader
# from tqdm import tqdm
from sklearn.decomposition import PCA
from pytorch_tabnet.metrics import Metric
from pytorch_tabnet.tab_model import TabNetRegressor
from torch.optim.lr_scheduler import CosineAnnealingWarmRestarts
import torch.optim as optim
import shutil
from sklearn.preprocessing import MinMaxScaler
# from tqdm import tqdm_notebook as tqdm
from darts.utils.statistics import check_seasonality, plot_acf
import darts.utils.timeseries_generation as tg
from darts.utils.timeseries_generation import datetime_attribute_timeseries
from darts.utils.missing_values import fill_missing_values
from darts.utils.likelihood_models import GaussianLikelihood

import logging
logging.disable(logging.CRITICAL)

%matplotlib inline


In [7]:

# DATA_DIR = '../input'
# DATA_DIR = './datasets'
DATA_DIR = '/Users/pujanmaharjan/uni adelaide/uofa_research_project/datasets'
# DATA_DIR = '/content/drive/My Drive/Research Project/datasets/'

# data configurations
# USE_PRECOMPUTE_FEATURES = True  # Load precomputed features for train.csv from private dataset (just for speed up)

# model & ensemble configurations
# PREDICT_CNN = True
# PREDICT_MLP = True
# PREDICT_GBDT = True
# PREDICT_TABNET = False

# GBDT_NUM_MODELS = 3
# GBDT_LR = 0.02  # 0.1

# NN_VALID_TH = 0.185
# NN_MODEL_TOP_N = 3
# TAB_MODEL_TOP_N = 3
ENSEMBLE_METHOD = 'mean'
# NN_NUM_MODELS = 10
# TABNET_NUM_MODELS = 5

# for saving quota
# IS_1ST_STAGE = True
# SHORTCUT_NN_IN_1ST_STAGE = True  # early-stop training to save GPU quota
# SHORTCUT_GBDT_IN_1ST_STAGE = True
# MEMORY_TEST_MODE = False

# for ablation studies
# CV_SPLIT = 'time'  # 'time': time-series KFold 'group': GroupKFold by stock-id
# USE_PRICE_NN_FEATURES = True  # Use nearest neighbor features that rely on tick size
# USE_VOL_NN_FEATURES = True  # Use nearest neighbor features that can be calculated without tick size
# USE_SIZE_NN_FEATURES = True  # Use nearest neighbor features that can be calculated without tick size
# USE_RANDOM_NN_FEATURES = False  # Use random index to aggregate neighbors

# USE_TIME_ID_NN = True  # Use time-id based neighbors
# USE_STOCK_ID_NN = True  # Use stock-id based neighbors

# ENABLE_RANK_NORMALIZATION = True  # Enable rank-normalization

EPOCHS = 10
SEED = 0


In [13]:
d = pd.read_parquet(os.path.join(DATA_DIR, 'optiver-realized-volatility-prediction', 'book_train.parquet', f'stock_id={0}'))

In [14]:
d.head()

,time_id,seconds_in_bucket,bid_price1,ask_price1,bid_price2,ask_price2,bid_size1,ask_size1,bid_size2,ask_size2
0,5,0,1.001422,1.002301,1.00137,1.002353,3,226,2,100
1,5,1,1.001422,1.002301,1.00137,1.002353,3,100,2,100
2,5,5,1.001422,1.002301,1.00137,1.002405,3,100,2,100
3,5,6,1.001422,1.002301,1.00137,1.002405,3,126,2,100
4,5,7,1.001422,1.002301,1.00137,1.002405,3,126,2,100


In [20]:
len(list(d['time_id'].unique()))

3830

In [21]:
d.groupby(by=["time_id"]).count()

,seconds_in_bucket,bid_price1,ask_price1,bid_price2,ask_price2,bid_size1,ask_size1,bid_size2,ask_size2
time_id,,,,,,,,,
5,302,302,302,302,302,302,302,302,302
11,200,200,200,200,200,200,200,200,200
16,188,188,188,188,188,188,188,188,188
31,120,120,120,120,120,120,120,120,120
62,176,176,176,176,176,176,176,176,176
...,...,...,...,...,...,...,...,...,...
32751,297,297,297,297,297,297,297,297,297
32753,206,206,206,206,206,206,206,206,206
32758,188,188,188,188,188,188,188,188,188


In [15]:
d.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 917553 entries, 0 to 917552
Data columns (total 10 columns):
 #   Column             Non-Null Count   Dtype  
---  ------             --------------   -----  
 0   time_id            917553 non-null  int16  
 1   seconds_in_bucket  917553 non-null  int16  
 2   bid_price1         917553 non-null  float32
 3   ask_price1         917553 non-null  float32
 4   bid_price2         917553 non-null  float32
 5   ask_price2         917553 non-null  float32
 6   bid_size1          917553 non-null  int32  
 7   ask_size1          917553 non-null  int32  
 8   bid_size2          917553 non-null  int32  
 9   ask_size2          917553 non-null  int32  
dtypes: float32(4), int16(2), int32(4)
memory usage: 31.5 MB


In [8]:
TIME_ID_ORDER_COL = 'time_id_order'
def split_by_time_id_order(data, split_size):
    if split_size < 0 and split_size >= 1:
        raise Exception("invalid split size. valid range (0,1)")
    # time_id_order ranges from 0 to 3829
    min_time_id_order = data[TIME_ID_ORDER_COL].min()
    max_time_id_order = data[TIME_ID_ORDER_COL].max()
    time_id_order_ix = int((max_time_id_order - min_time_id_order) * split_size)
    time_id_order_to_split = min_time_id_order + time_id_order_ix
    data_copy = data.copy()

    first_part = data_copy[data_copy[TIME_ID_ORDER_COL] <= time_id_order_to_split]
    second_part = data_copy[data_copy[TIME_ID_ORDER_COL] > time_id_order_to_split]
    return first_part, second_part

def split_df_into_train_val_test(data, train_size, val_size):
    train_df, val_test_df = split_by_time_id_order(data, train_size)
    val_df, test_df = split_by_time_id_order(val_test_df, val_size)

    return train_df, val_df, test_df

In [9]:
# def split_df_into_train_test(df):
#     train_index = int(len(df) * 0.8)
#     train_data = df[:train_index]
#     test_data = df[train_index:]
#     # print('Train data shape ', train_data.shape)
#     # print('Test data shape ', test_data.shape)
#     return train_data, test_data

# def split_df_into_train_val_test(df):
#     # split 70, 15, 15
#     train_index = int(len(df) * 0.7)
#     train_data = df[:train_index]
#     val_test_data = df[train_index:]
#     val_index = int(len(val_test_data) * 0.5)
#     val_data = val_test_data[:val_index]
#     test_data = val_test_data[val_index:]
#     # print('Total data shape ', df.shape)
#     # print('train shape ', train_data.shape)
#     # print('validation shape ', val_data.shape)
#     # print('test shape ', test_data.shape)
#     return train_data, val_data, test_data

@contextmanager
def timer(name: str):
    s = time.time()
    yield
    elapsed = time.time() - s
    print(f'[{name}] {elapsed: .3f}sec')

def print_trace(name: str = ''):
    print(f'ERROR RAISED IN {name or "anonymous"}')
    print(traceback.format_exc())



## Feature Engineering

### Base Features

In [11]:
def make_rv_lags(df_rv_lag, number_of_periods: int):
    # add past rv
    for i in range(1, number_of_periods + 1):
        df_rv_lag[f'lag_{i}_rv'] = df_rv_lag['target'].shift(periods=i)

    df_rv_lag = df_rv_lag.dropna()
    return df_rv_lag

In [10]:
class DataBlock(Enum):
    TRAIN = 1
    TEST = 2
    BOTH = 3

def load_stock_data(stock_id: int, directory: str) -> pd.DataFrame:
    return pd.read_parquet(os.path.join(DATA_DIR, 'optiver-realized-volatility-prediction', directory, f'stock_id={stock_id}'))

def load_data(stock_id: int, stem: str, block: DataBlock) -> pd.DataFrame:
    if block == DataBlock.TRAIN:
        return load_stock_data(stock_id, f'{stem}_train.parquet')
    elif block == DataBlock.TEST:
        return load_stock_data(stock_id, f'{stem}_test.parquet')
    else:
        return pd.concat([
            load_data(stock_id, stem, DataBlock.TRAIN),
            load_data(stock_id, stem, DataBlock.TEST)
        ]).reset_index(drop=True)

def load_book(stock_id: int, block: DataBlock=DataBlock.TRAIN) -> pd.DataFrame:
    return load_data(stock_id, 'book', block)

def load_trade(stock_id: int, block=DataBlock.TRAIN) -> pd.DataFrame:
    return load_data(stock_id, 'trade', block)

def calc_wap1(df: pd.DataFrame) -> pd.Series:
    wap = (df['bid_price1'] * df['ask_size1'] + df['ask_price1'] * df['bid_size1']) / (df['bid_size1'] + df['ask_size1'])
    return wap

def calc_wap2(df: pd.DataFrame) -> pd.Series:
    wap = (df['bid_price2'] * df['ask_size2'] + df['ask_price2'] * df['bid_size2']) / (df['bid_size2'] + df['ask_size2'])
    return wap

def realized_volatility(series):
    return np.sqrt(np.sum(series**2))

def log_return(series: np.ndarray):
    return np.log(series).diff()

def log_return_df2(series: np.ndarray):
    return np.log(series).diff(2)

def flatten_name(prefix, src_names):
    ret = []
    for c in src_names:
        if c[0] in ['time_id', 'stock_id']:
            ret.append(c[0])
        else:
            ret.append('.'.join([prefix] + list(c)))
    return ret

def make_book_feature(stock_id, 
                      block,
                      add_spread_features,
                      add_statistics_features,
                      add_book_time_features):
    book = load_book(stock_id, block)

    book['wap1'] = calc_wap1(book)
    book['wap2'] = calc_wap2(book)
    book['log_return1'] = book.groupby(['time_id'], group_keys=False)['wap1'].apply(log_return)
    book['log_return2'] = book.groupby(['time_id'], group_keys=False)['wap2'].apply(log_return)
    book['log_return_ask1'] = book.groupby(['time_id'], group_keys=False)['ask_price1'].apply(log_return)
    book['log_return_ask2'] = book.groupby(['time_id'], group_keys=False)['ask_price2'].apply(log_return)
    book['log_return_bid1'] = book.groupby(['time_id'], group_keys=False)['bid_price1'].apply(log_return)
    book['log_return_bid2'] = book.groupby(['time_id'], group_keys=False)['bid_price2'].apply(log_return)

    if add_spread_features:
        book['wap_balance'] = abs(book['wap1'] - book['wap2'])
        book['price_spread'] = (book['ask_price1'] - book['bid_price1']) / ((book['ask_price1'] + book['bid_price1']) / 2)
        book['bid_spread'] = book['bid_price1'] - book['bid_price2']
        book['ask_spread'] = book['ask_price1'] - book['ask_price2']
        book['total_volume'] = (book['ask_size1'] + book['ask_size2']) + (book['bid_size1'] + book['bid_size2'])
        book['volume_imbalance'] = abs((book['ask_size1'] + book['ask_size2']) - (book['bid_size1'] + book['bid_size2']))

    features = {
        'wap1': [np.sum],
        'wap2': [np.sum],
        'log_return1': [realized_volatility],
        'log_return2': [realized_volatility],
        'log_return_ask1': [realized_volatility],
        'log_return_ask2': [realized_volatility],
        'log_return_bid1': [realized_volatility],
        'log_return_bid2': [realized_volatility],
    }

    if add_spread_features and add_statistics_features:
        features = {
            'seconds_in_bucket': ['count'],
            'wap1': [np.sum, np.mean, np.std],
            'wap2': [np.sum, np.mean, np.std],
            'log_return1': [np.sum, realized_volatility, np.mean, np.std],
            'log_return2': [np.sum, realized_volatility, np.mean, np.std],
            'log_return_ask1': [np.sum, realized_volatility, np.mean, np.std],
            'log_return_ask2': [np.sum, realized_volatility, np.mean, np.std],
            'log_return_bid1': [np.sum, realized_volatility, np.mean, np.std],
            'log_return_bid2': [np.sum, realized_volatility, np.mean, np.std],
            'wap_balance': [np.sum, np.mean, np.std],
            'price_spread':[np.sum, np.mean, np.std],
            'bid_spread':[np.sum, np.mean, np.std],
            'ask_spread':[np.sum, np.mean, np.std],
            'total_volume':[np.sum, np.mean, np.std],
            'volume_imbalance':[np.sum, np.mean, np.std]
        }
    elif add_spread_features and not add_statistics_features:
        features = {
            'seconds_in_bucket': ['count'],
            'wap1': [np.sum],
            'wap2': [np.sum],
            'log_return1': [realized_volatility],
            'log_return2': [realized_volatility],
            'log_return_ask1': [np.sum, realized_volatility],
            'log_return_ask2': [np.sum, realized_volatility],
            'log_return_bid1': [np.sum, realized_volatility],
            'log_return_bid2': [np.sum, realized_volatility],
            'wap_balance': [np.sum],
            'price_spread':[np.sum],
            'bid_spread':[np.sum],
            'ask_spread':[np.sum],
            'total_volume':[np.sum],
            'volume_imbalance':[np.sum]
        }

    agg = book.groupby('time_id', group_keys=False).agg(features).reset_index(drop=False)
    agg.columns = flatten_name('book', agg.columns)
    agg['stock_id'] = stock_id

    if add_book_time_features:
        for time in [450, 300, 150]:
            d = book[book['seconds_in_bucket'] >= time].groupby('time_id', group_keys=False).agg(features).reset_index(drop=False)
            d.columns = flatten_name(f'book_{time}', d.columns)
            agg = pd.merge(agg, d, on='time_id', how='left')
    return agg

def make_trade_feature(stock_id, block, add_trade_time_features):
    trade = load_trade(stock_id, block)
    trade['log_return'] = trade.groupby('time_id', group_keys=False)['price'].apply(log_return)

    features = {
        'log_return':[realized_volatility],
        'seconds_in_bucket':['count'],
        'size':[np.sum],
        'order_count':[np.mean],
    }

    agg = trade.groupby('time_id', group_keys=False).agg(features).reset_index()
    agg.columns = flatten_name('trade', agg.columns)
    agg['stock_id'] = stock_id

    if add_trade_time_features:
        for time in [450, 300, 150]:
            d = trade[trade['seconds_in_bucket'] >= time].groupby('time_id').agg(features).reset_index(drop=False)
            d.columns = flatten_name(f'trade_{time}', d.columns)
            agg = pd.merge(agg, d, on='time_id', how='left')
    return agg

def make_book_feature_v2(stock_id, block):
    book = load_book(stock_id, block)

    prices = book.set_index('time_id')[['bid_price1', 'ask_price1', 'bid_price2', 'ask_price2']]
    time_ids = list(set(prices.index))

    ticks = {}
    for tid in time_ids:
        try:
            price_list = prices.loc[tid].values.flatten()
            price_diff = sorted(np.diff(sorted(set(price_list))))
            ticks[tid] = price_diff[0]
        except Exception:
            print_trace(f'tid={tid}')
            ticks[tid] = np.nan

    dst = pd.DataFrame()
    dst['time_id'] = np.unique(book['time_id'])
    dst['stock_id'] = stock_id
    dst['tick_size'] = dst['time_id'].map(ticks)

    return dst
    

def make_features(base, block, add_spread_features, add_statistics_features, add_book_time_features, add_trade_time_features):
    stock_ids = set(base['stock_id'])
    with timer('books'):
        books = Parallel(n_jobs=-1)(delayed(make_book_feature)(stock_id, block, add_spread_features, add_statistics_features, add_book_time_features) for stock_id in stock_ids)
        book = pd.concat(books)

    with timer('trades'):
        trades = Parallel(n_jobs=-1)(delayed(make_trade_feature)(stock_id, block, add_trade_time_features) for stock_id in stock_ids)
        trade = pd.concat(trades)

    with timer('extra features'):
        df = pd.merge(base, book, on=['stock_id', 'time_id'], how='left')
        df = pd.merge(df, trade, on=['stock_id', 'time_id'], how='left')

    df = make_rv_lags(df, 1)
    
    return df

def make_features_tick_size(base, block):
    stock_ids = set(base['stock_id'])
    with timer('books(v2)'):
        books = Parallel(n_jobs=-1)(delayed(make_book_feature_v2)(i, block) for i in stock_ids)
        book_v2 = pd.concat(books)

    d = pd.merge(base, book_v2, on=['stock_id', 'time_id'], how='left')
    return d

In [82]:
# book_df = load_book(0, DataBlock.TRAIN)
# book_df

In [83]:
# book_train_df = make_book_feature(0, DataBlock.TRAIN)
# book_train_df

In [84]:
# trade_df = load_trade(0, DataBlock.TRAIN)
# trade_df

In [ ]:
dd = pd.read_csv("./data/order_trade_data_0.csv")
dd

In [ ]:
dd2 = pd.read_csv("./data/order_trade_agg.csv")
dd2

In [85]:
# trade_train_df = make_trade_feature(0, DataBlock.TRAIN)
# trade_train_df

### Nearest-Neighbor Features

In [8]:
N_NEIGHBORS_MAX = 80

from pandas import DataFrame

class Neighbors:
    def __init__(self,
                 n_neighbors: int,
                 name: str,
                 pivot: pd.DataFrame,
                 p: float,
                 metric: str = 'minkowski',
                 metric_params: Optional[Dict] = None,
                 exclude_self: bool = False):
        self.name = name
        self.exclude_self = exclude_self
        self.p = p
        self.metric = metric
        self.n_neighbors = n_neighbors

        if metric == 'random':
            n_queries = len(pivot)
            self.neighbors = np.random.randint(n_queries, size=(n_queries, n_neighbors))
        else:
            print('metric ', metric)

            nn = NearestNeighbors(
                n_neighbors=n_neighbors,
                p=p,
                metric=metric,
                metric_params=metric_params
            )

            nn.fit(pivot)
            _, self.neighbors = nn.kneighbors(pivot, return_distance=True)

        self.columns = self.index = self.feature_values = self.feature_col = None

    def rearrange_feature_values(self, df: pd.DataFrame, feature_col: str) -> None:
        raise NotImplementedError()

    def make_nn_feature(self, n=5, agg=np.mean) -> pd.DataFrame:
        assert self.feature_values is not None, "should call rearrange_feature_values beforehand"

        start = 1 if self.exclude_self else 0

        pivot_aggs = pd.DataFrame(
            agg(self.feature_values[start:n,:,:], axis=0),
            columns=self.columns,
            index=self.index
        )

        dst = pivot_aggs.unstack().reset_index()
        dst.columns = ['stock_id', 'time_id', f'{self.feature_col}_nn{n}_{self.name}_{agg.__name__}']
        # print('Destination columns ', dst.columns)
        return dst


class TimeIdNeighbors(Neighbors):
    def __init__(self, n_neighbors: int, name: str, pivot: DataFrame, p: float, metric: str = 'minkowski', metric_params: Dict | None = None, exclude_self: bool = False):
        super().__init__(n_neighbors, name, pivot, p, metric, metric_params, exclude_self)

    def rearrange_feature_values(self, df: pd.DataFrame, feature_col: str) -> None:
        feature_pivot = df.pivot('time_id', 'stock_id', feature_col)
        feature_pivot = feature_pivot.fillna(feature_pivot.mean())
        feature_pivot.head()

        feature_values = np.zeros((self.n_neighbors, *feature_pivot.shape))

        for i in range(self.n_neighbors):
            feature_values[i, :, :] += feature_pivot.values[self.neighbors[:, i], :]

        self.columns = list(feature_pivot.columns)
        self.index = list(feature_pivot.index)
        self.feature_values = feature_values
        self.feature_col = feature_col

    def __repr__(self) -> str:
        return f"time-id NN (name={self.name}, metric={self.metric}, p={self.p})"


class StockIdNeighbors(Neighbors):
    def __init__(self, n_neighbors: int, name: str, pivot: DataFrame, p: float, metric: str = 'minkowski', metric_params: Dict | None = None, exclude_self: bool = False):
        super().__init__(n_neighbors, name, pivot, p, metric, metric_params, exclude_self)

    def rearrange_feature_values(self, df: pd.DataFrame, feature_col: str) -> None:
        """stock-id based nearest neighbor features"""
        feature_pivot = df.pivot('time_id', 'stock_id', feature_col)
        feature_pivot = feature_pivot.fillna(feature_pivot.mean())

        feature_values = np.zeros((self.n_neighbors, *feature_pivot.shape))

        for i in range(self.n_neighbors):
            feature_values[i, :, :] += feature_pivot.values[:, self.neighbors[:, i]]

        self.columns = list(feature_pivot.columns)
        self.index = list(feature_pivot.index)
        self.feature_values = feature_values
        self.feature_col = feature_col

    def __repr__(self) -> str:
        return f"stock-id NN (name={self.name}, metric={self.metric}, p={self.p})"


In [87]:
# add_tau_features
# the tau itself is meaningless for GBDT, but useful as input to aggregate in Nearest Neighbor features
def add_tau_features(df_tau):
    df_tau['trade.tau'] = np.sqrt(1 / df_tau['trade.seconds_in_bucket.count'])
    df_tau['trade_150.tau'] = np.sqrt(1 / df_tau['trade_150.seconds_in_bucket.count'])
    df_tau['book.tau'] = np.sqrt(1 / df_tau['book.seconds_in_bucket.count'])
    df_tau['real_price'] = 0.01 / df_tau['tick_size']

    return df_tau

#### Build Nearest Neighbors

In [88]:
# build_nearest_neighbors
def build_nearest_neighbors(df_nn,
                            n_neighbors: int,
                            use_price_nn_features: bool,
                            use_volume_nn_features: bool,
                            use_size_nn_features: bool,
                            use_random_nn_features: bool):
    time_id_neighbors: List[Neighbors] = []
    stock_id_neighbors: List[Neighbors] = []

    with timer('knn fit'):
        df_pv = df_nn[['stock_id', 'time_id']].copy()
        df_pv['price'] = 0.01 / df_nn['tick_size']
        df_pv['vol'] = df_nn['book.log_return1.realized_volatility']
        df_pv['trade.tau'] = df_nn['trade.tau']
        df_pv['trade.size.sum'] = df_nn['book.total_volume.sum']

        print('USE_PRICE_NN_FEATURES ', use_price_nn_features)
        if use_price_nn_features:
            pivot = df_pv.pivot('time_id', 'stock_id', 'price')
            pivot = pivot.fillna(pivot.mean())
            pivot = pd.DataFrame(minmax_scale(pivot))

            time_id_neighbors.append(
                TimeIdNeighbors(
                    n_neighbors,
                    'time_price_c',
                    pivot,
                    p=2,
                    metric='canberra',
                    exclude_self=True
                )
            )
            time_id_neighbors.append(
                TimeIdNeighbors(
                    n_neighbors,
                    'time_price_m',
                    pivot,
                    p=2,
                    metric='mahalanobis',
                    metric_params={'VI':np.linalg.inv(np.cov(pivot.values.T))}
                )
            )
            stock_id_neighbors.append(
                StockIdNeighbors(
                    n_neighbors,
                    'stock_price_l1',
                    minmax_scale(pivot.transpose()),
                    p=1,
                    exclude_self=True)
            )

        print('USE_VOL_NN_FEATURES ', use_volume_nn_features)
        if use_volume_nn_features:
            pivot = df_pv.pivot('time_id', 'stock_id', 'vol')
            pivot = pivot.fillna(pivot.mean())
            pivot = pd.DataFrame(minmax_scale(pivot))

            time_id_neighbors.append(
                TimeIdNeighbors(n_neighbors, 'time_vol_l1', pivot, p=1)
            )
            stock_id_neighbors.append(
                StockIdNeighbors(
                    n_neighbors,
                    'stock_vol_l1',
                    minmax_scale(pivot.transpose()),
                    p=1,
                    exclude_self=True
                )
            )

        print('USE_SIZE_NN_FEATURES ', use_size_nn_features)
        if use_size_nn_features:
            pivot = df_pv.pivot('time_id', 'stock_id', 'trade.size.sum')
            pivot = pivot.fillna(pivot.mean())
            pivot = pd.DataFrame(minmax_scale(pivot))

            time_id_neighbors.append(
                TimeIdNeighbors(
                    n_neighbors,
                    'time_size_m',
                    pivot,
                    p=2,
                    metric='mahalanobis',
                    # metric_params={'V':np.cov(pivot.values.T)}
                    metric_params={'VI':np.linalg.inv(np.cov(pivot.values.T))}
                )
            )
            time_id_neighbors.append(
                TimeIdNeighbors(
                    n_neighbors,
                    'time_size_c',
                    pivot,
                    p=2,
                    metric='canberra'
                )
            )

        print('USE_RANDOM_NN_FEATURES ', use_random_nn_features)
        if use_random_nn_features:
            pivot = df_pv.pivot('time_id', 'stock_id', 'vol')
            pivot = pivot.fillna(pivot.mean())
            pivot = pd.DataFrame(minmax_scale(pivot))

            time_id_neighbors.append(
                TimeIdNeighbors(
                    n_neighbors,
                    'time_random',
                    pivot,
                    p=2,
                    metric='random'
                )
            )
            stock_id_neighbors.append(
                StockIdNeighbors(
                    n_neighbors,
                    'stock_random',
                    pivot.transpose(),
                    p=2,
                    metric='random')
            )

    return time_id_neighbors, stock_id_neighbors

#### Check Neighbor Indices

In [89]:
# calculate_rank_correraltion
def calculate_rank_correraltion(neighbors, top_n=5):
    if not neighbors:
        return
    neighbor_indices = pd.DataFrame()
    for n in neighbors:
        neighbor_indices[n.name] = n.neighbors[:,:top_n].flatten()

    sns.heatmap(neighbor_indices.corr('kendall'), annot=True)

In [90]:
# display_neighbors
def display_neighbors(df_neighbor, neighbors_to_display, column_name, number_of_neighbor):
    ids = np.array(sorted(df_neighbor[column_name].unique()))
    for neighbor in neighbors_to_display:
        print(neighbor)
        display(
            pd.DataFrame(
                ids[neighbor.neighbors[:,:number_of_neighbor]],
                index=pd.Index(ids, name=column_name),
                # ALERT: NOTE value was 10 in range and was updated to 2
                columns=[f'top_{i+1}' for i in range(number_of_neighbor)] #10
            ).iloc[1:6]
        )

In [91]:
# display_neighbors(df, time_id_neighbors, 'time_id', 10)

In [92]:
# display_neighbors(df, stock_id_neighbors, 'stock_id', 10)

In [93]:

# stock_ids = np.array(sorted(df['stock_id'].unique()))
# for neighbor in stock_id_neighbors:
#     print(neighbor)
#     display(
#         pd.DataFrame(
#             stock_ids[neighbor.neighbors[:,:10]],
#             index=pd.Index(stock_ids, name='stock_id'),
#             # NOTE: range was 10,
#             columns=[f'top_{i+1}' for i in range(10)] #10
#         ).loc[0] #64
#     )


In [94]:
# calculate_rank_correraltion(time_id_neighbors)

In [95]:
# calculate_rank_correraltion(stock_id_neighbors)

#### Aggregate Features With Neighbors

In [96]:
# aggregate_features_with_neighbors
# features with large changes over time are converted to relative ranks within time-id
def aggregate_features_with_neighbors(df_agg):
    df_agg['trade.order_count.mean'] = df_agg.groupby('time_id', group_keys=False)['trade.order_count.mean'].rank()
    df_agg['book.total_volume.sum']  = df_agg.groupby('time_id', group_keys=False)['book.total_volume.sum'].rank()
    df_agg['book.total_volume.mean'] = df_agg.groupby('time_id', group_keys=False)['book.total_volume.mean'].rank()
    df_agg['book.total_volume.std']  = df_agg.groupby('time_id')['book.total_volume.std'].rank()

    df_agg['trade.tau'] = df_agg.groupby('time_id', group_keys=False)['trade.tau'].rank()

    for dt in [150, 300, 450]:
        df_agg[f'book_{dt}.total_volume.sum']  = df_agg.groupby('time_id', group_keys=False)[f'book_{dt}.total_volume.sum'].rank()
        df_agg[f'book_{dt}.total_volume.mean'] = df_agg.groupby('time_id', group_keys=False)[f'book_{dt}.total_volume.mean'].rank()
        df_agg[f'book_{dt}.total_volume.std']  = df_agg.groupby('time_id', group_keys=False)[f'book_{dt}.total_volume.std'].rank()
        df_agg[f'trade_{dt}.order_count.mean'] = df_agg.groupby('time_id', group_keys=False)[f'trade_{dt}.order_count.mean'].rank()

    return df_agg

In [97]:
# make_nearest_neighbor_feature
def make_nearest_neighbor_feature(df_nn: pd.DataFrame, time_id_neighbors, stock_id_neighbors, use_price_nn_features) -> pd.DataFrame:
    df_nnf = df_nn.copy()

    feature_cols_stock = {
        'book.log_return1.realized_volatility': [np.mean, np.min, np.max, np.std],
        'trade.seconds_in_bucket.count': [np.mean],
        'trade.tau': [np.mean],
        'trade_150.tau': [np.mean],
        'book.tau': [np.mean],
        'trade.size.sum': [np.mean],
        'book.seconds_in_bucket.count': [np.mean],
    }

    feature_cols = {
        'book.log_return1.realized_volatility': [np.mean, np.min, np.max, np.std],
        'real_price': [np.max, np.mean, np.min],
        'trade.seconds_in_bucket.count': [np.mean],
        'trade.tau': [np.mean],
        'trade.size.sum': [np.mean],
        'book.seconds_in_bucket.count': [np.mean],
        'trade_150.tau_nn20_stock_vol_l1_mean': [np.mean],
        'trade.size.sum_nn20_stock_vol_l1_mean': [np.mean],
    }

    time_id_neigbor_sizes = [3, 5, 10, 20, 40]
    time_id_neigbor_sizes_vol = [2, 3, 5, 10, 20, 40]
    stock_id_neighbor_sizes = [10, 20, 40]

    ndf: Optional[pd.DataFrame] = None

    def _add_ndf(ndf: Optional[pd.DataFrame], dst: pd.DataFrame) -> pd.DataFrame:
        if ndf is None:
            return dst
        else:
            ndf[dst.columns[-1]] = dst[dst.columns[-1]].astype(np.float32)
            return ndf

    # neighbor stock_id
    for feature_col in feature_cols_stock.keys():
        # print('Feature column ', feature_col)
        try:
            if feature_col not in df_nnf.columns:
                print(f"column {feature_col} is skipped")
                continue

            if not stock_id_neighbors:
                continue

            for nn in stock_id_neighbors:
                nn.rearrange_feature_values(df_nnf, feature_col)

            for agg in feature_cols_stock[feature_col]:
                for n in stock_id_neighbor_sizes:
                    try:
                        for nn in stock_id_neighbors:
                            dst = nn.make_nn_feature(n, agg)
                            ndf = _add_ndf(ndf, dst)
                    except Exception:
                        print_trace('stock-id nn')
                        pass
        except Exception:
            print_trace('stock-id nn')
            pass

    if ndf is not None:
        df_nnf = pd.merge(df_nnf, ndf, on=['time_id', 'stock_id'], how='left')
    ndf = None

    # neighbor time_id
    for feature_col in feature_cols.keys():
        # print('Feature col ', feature_col)
        try:
            # if feature_col == 'real_price':
            #     continue
            if feature_col not in df_nnf.columns:
                print(f"column {feature_col} is skipped")
                continue

            for nn in time_id_neighbors:
                # print('calling rearrange_feature_values for feature_col ', feature_col)
                nn.rearrange_feature_values(df_nnf, feature_col)

            if 'volatility' in feature_col:
                time_id_ns = time_id_neigbor_sizes_vol
            else:
                time_id_ns = time_id_neigbor_sizes

            for agg in feature_cols[feature_col]:
                for n in time_id_ns:
                    try:
                        for nn in time_id_neighbors:
                            # print('calling make_nn_feature for feature_col ', feature_col)
                            dst = nn.make_nn_feature(n, agg)
                            ndf = _add_ndf(ndf, dst)
                    except Exception:
                        print_trace('Exception in time-id nn in make_nn_feature ', feature_col)
                        pass
        except Exception:
            print_trace('Exception in neighbor time-id nn ', feature_col)

    if ndf is not None:
        df_nnf = pd.merge(df_nnf, ndf, on=['time_id', 'stock_id'], how='left')

    # features further derived from nearest neighbor features
    try:
        if use_price_nn_features:
            for sz in time_id_neigbor_sizes:
                denominator = f"real_price_nn{sz}_time_price_c"

                df_nnf[f'real_price_rankmin_{sz}']  = df_nnf['real_price'] / df_nnf[f"{denominator}_amin"]
                df_nnf[f'real_price_rankmax_{sz}']  = df_nnf['real_price'] / df_nnf[f"{denominator}_amax"]
                df_nnf[f'real_price_rankmean_{sz}'] = df_nnf['real_price'] / df_nnf[f"{denominator}_mean"]

            for sz in time_id_neigbor_sizes_vol:
                denominator = f"book.log_return1.realized_volatility_nn{sz}_time_price_c"

                df_nnf[f'vol_rankmin_{sz}'] = \
                    df_nnf['book.log_return1.realized_volatility'] / df_nnf[f"{denominator}_amin"]
                df_nnf[f'vol_rankmax_{sz}'] = \
                    df_nnf['book.log_return1.realized_volatility'] / df_nnf[f"{denominator}_amax"]

        price_cols = [c for c in df_nnf.columns if 'real_price' in c and 'rank' not in c]
        for c in price_cols:
            del df_nnf[c]

        if use_price_nn_features:
            for sz in time_id_neigbor_sizes_vol:
                tgt = f'book.log_return1.realized_volatility_nn{sz}_time_price_m_mean'
                df_nnf[f'{tgt}_rank'] = df_nnf.groupby('time_id', group_keys=False)[tgt].rank()
    except Exception:
        print_trace('nn features')

    return df_nnf

### Misc Features

In [98]:
# skew correction for NN
def skew_correction_for_nn(df_skew):
    cols_to_log = [
        'trade.size.sum',
        'trade_150.size.sum',
        'trade_300.size.sum',
        'trade_450.size.sum',
        'volume_imbalance'
    ]
    for c in df_skew.columns:
        for check in cols_to_log:
            try:
                if check in c:
                    df_skew[c] = np.log(df_skew[c]+1)
                    break
            except Exception:
                print_trace('log1p')

    return df_skew

In [99]:
# Rolling average of RV for similar trading volume
def rolling_average_of_rv_for_similar_trading_volume(df_ra):
    try:
        df_ra.sort_values(by=['stock_id', 'book.total_volume.sum'], inplace=True)
        df_ra.reset_index(drop=True, inplace=True)

        roll_target = 'book.log_return1.realized_volatility'

        for window_size in [3, 10]:
            df_ra[f'realized_volatility_roll{window_size}_by_book.total_volume.mean'] = \
                df_ra.groupby('stock_id', group_keys=False)[roll_target].rolling(window_size, center=True, min_periods=1) \
                                                    .mean() \
                                                    .reset_index() \
                                                    .sort_values(by=['level_1'])[roll_target].values
    except Exception:
        print_trace('mean RV')

    return df_ra

In [100]:
# # stock-id embedding (helps little)
# try:
#     lda_n = 3
#     lda = LatentDirichletAllocation(n_components=lda_n, random_state=0)

#     stock_id_emb = pd.DataFrame(
#         lda.fit_transform(pivot.transpose()),
#         index=df_pv.pivot('time_id', 'stock_id', 'vol').columns
#     )

#     for i in range(lda_n):
#         df2[f'stock_id_emb{i}'] = df2['stock_id'].map(stock_id_emb[i])
# except Exception:
#     print_trace('LDA')

## Reverse Engineering time-id Order & Make CV Split

In [101]:
# reverse engineering time-id order
%matplotlib inline

@contextmanager
def timer(name):
    s = time.time()
    yield
    e = time.time() - s
    print(f"[{name}] {e:.3f}sec")

def calc_price2(df):
    tick = sorted(np.diff(sorted(np.unique(df.values.flatten()))))[0]
    return 0.01 / tick

def calc_prices(r):
    df = pd.read_parquet(r.book_path, columns=['time_id', 'ask_price1', 'ask_price2', 'bid_price1', 'bid_price2'])
    df = df.set_index('time_id')
    df = df.groupby(level='time_id', group_keys=False).apply(calc_price2).to_frame('price').reset_index()
    df['stock_id'] = r.stock_id
    return df

def sort_manifold(df, clf):
    df_ = df.set_index('time_id')
    df_ = pd.DataFrame(minmax_scale(df_.fillna(df_.mean())))

    X_compoents = clf.fit_transform(df_)

    dft = df.reindex(np.argsort(X_compoents[:,0])).reset_index(drop=True)
    return np.argsort(X_compoents[:, 0]), X_compoents

def reconstruct_time_id_order():
    print('reconstruct_time_id_order started. loading files')
    with timer('load files'):
        book_path = DATA_DIR + '/optiver-realized-volatility-prediction/book_train.parquet/**/*.parquet'
        print('book path ', book_path)
        df_files = pd.DataFrame(
            {'book_path': glob.glob(book_path)}).eval('stock_id = book_path.str.extract("stock_id=(\d+)").astype("int")', engine='python')

    print('reconstruct_time_id_order calculating prices')
    with timer('calc prices'):
        df_prices = pd.concat(Parallel(n_jobs=4, verbose=51)(delayed(calc_prices)(r) for _, r in df_files.iterrows()))
        df_prices = df_prices.pivot('time_id', 'stock_id', 'price')
        df_prices.columns = [f'stock_id={i}' for i in df_prices.columns]
        df_prices = df_prices.reset_index(drop=False)

    print('reconstruct_time_id_order tSNE')
    with timer('t-SNE(400) -> 50'):
        clf = TSNE(n_components=1, perplexity=400, random_state=0, n_iter=2000)
        order, X_compoents = sort_manifold(df_prices, clf)

        clf = TSNE(n_components=1, perplexity=50, random_state=0, init=X_compoents, n_iter=2000, method='exact')
        order, X_compoents = sort_manifold(df_prices, clf)

        df_ordered = df_prices.reindex(order).reset_index(drop=True)
        if df_ordered['stock_id=0'].iloc[0] > df_ordered['stock_id=0'].iloc[-1]:
            df_ordered = df_ordered.reindex(df_ordered.index[::-1]).reset_index(drop=True)

    # AMZN
    # plt.plot(df_ordered['stock_id=61'])

    return df_ordered[['time_id']]

In [102]:
# # add_time_id_order
# def add_time_id_order(df_tid):
#     print('add_time_id_order')
#     timeid_order = reconstruct_time_id_order()
#     print('time_order received ')
#     timeid_order['time_id_order'] = np.arange(len(timeid_order))
#     df_tid['time_id_order'] = df_tid['time_id'].map(timeid_order.set_index('time_id')['time_id_order'])
#     df_tid = df_tid.sort_values(['time_id_order', 'stock_id']).reset_index(drop=True)
#     df_tid.reset_index(drop=True, inplace=True)

#     return df_tid

In [103]:
# chek_null_columns
def chek_null_columns(X):
    xp = X.isna().any()
    xp_null = xp.loc[lambda x : x == True]
    nan_columns = list(xp_null.index)
    print('Null columns ', nan_columns)
    # X = X.drop(columns=nan_columns)
    # return X

In [104]:
# plot_time_id_ordered
def plot_time_id_ordered(stock_id, df, first_n_records = None):
    df_train_per_stock = df[df['stock_id'] == stock_id]
    if first_n_records:
        df_train_per_stock = df_train_per_stock[0: first_n_records]
    print('df_train_per_stock.shape',df_train_per_stock.shape)
    plt.plot(range(len(df_train_per_stock)), df_train_per_stock['target'])
    plt.title('Time Id ordered plot of target')
    plt.xlabel('Time')
    plt.ylabel('Realized volatility')
    plt.title('Reealized volatility for stock ' + str(stock_id))


In [105]:
# plot_time_id_ordered_plot(0, df_train, 36*2)

In [106]:
# plot_time_id_ordered_plot(0, df_train)

In [107]:
model_results = []

In [108]:
# modal results operations
def get_model_results_df():
    return pd.DataFrame(model_results)

def reset_model_results():
    model_results = []

def rmspe_error(y_true, y_pred):
    return  np.sqrt(np.mean(np.square((y_true - y_pred) / y_true)))

def add_model_result(model_name: str, y_true, y_pred, isDart: bool, feature: str, 
    time_taken: datetime, learning_rate: float, epochs: int,
    stock_ids: list[int]):
    if y_true is None:
        print('y_true is none. model_name ', model_name, ', feature ', feature, ', epoch ', epochs, ', learning rate ', learning_rate)
        return

    if y_pred is None:
        print('y_pred is none. model_name ', model_name, ', feature ', feature, ', epoch ', epochs, ', learning rate ', learning_rate)
        return

    if isDart:
        mae_value = mae(y_true, y_pred)
        rmse_value = rmse(y_true, y_pred)
        mse_value = mse(y_true, y_pred)
        mape_value = None
        rmspe_value = None
        # mape_value = mape(y_true, y_pred)
        # y_true_df = y_true.pd_dataframe().to_numpy()
        # y_pred_df = y_pred.pd_dataframe().to_numpy()
        # print('calculating rmspe for dart')
        # rmspe_value = rmspe_error(y_true_df, y_pred_df)
        # print('rmspe value ', rmspe_value)
    else:
        mse_value = mean_squared_error(y_true, y_pred)
        rmse_value = mean_squared_error(y_true, y_pred, squared=False)
        mae_value = mean_absolute_error(y_true, y_pred)
        rmspe_value = rmspe_error(y_true, y_pred)
        mape_value = mean_absolute_percentage_error(y_true, y_pred)
   

    # model_result_existing = [m for m in model_results if
    #                          m['model_name'].lower() == model_name.lower()
    #                          and m['feature'].lower() == feature.lower()
    #                          and m['learning_rate'] == learning_rate
    #                          and m['epochs'] = epochs]
    # if model_result_existing:
    #     for model in model_results:
    #         if model['model_name'] == model_name:
    #             model['mse'] = mse_value
    #             model['mae'] = mae_value
    #             model['rmse'] = rmse_value
    #             model['added_date'] = datetime.now()
    #             model['time_taken'] = time_taken
    #             model['learning_rate'] = learning_rate
    #             model['epochs'] = epochs
    # else:
    model_results.append({'model_name': model_name,
                            'mse': mse_value,
                            'rmse': rmse_value,
                            'mae': mae_value,
                            'rmspe': rmspe_value,
                            'mape': mape_value,
                            'added_date': datetime.now(),
                            'feature': feature,
                            'time_taken': time_taken.total_seconds(),
                            'learning_rate': learning_rate,
                            'epochs': epochs,
                            'stock_ids': stock_ids,
                            'stock_ids_count': len(stock_ids)
                            })
    

    df_res = get_model_results_df()
    df_res.to_csv("./data/model_results.csv", index=False)

    return model_results

## LightGBM Training

In [109]:
# light gbm
def rmspe(y_true, y_pred):
    return  (np.sqrt(np.mean(np.square((y_true - y_pred) / y_true))))

def feval_RMSPE(preds, train_data):
    labels = train_data.get_label()
    return 'RMSPE', round(rmspe(y_true = labels, y_pred = preds),5), False

# from: https://blog.amedama.jp/entry/lightgbm-cv-feature-importance
def plot_importance(cvbooster, figsize=(10, 10)):
    raw_importances = cvbooster.feature_importance(importance_type='gain')
    feature_name = cvbooster.boosters[0].feature_name()
    importance_df = pd.DataFrame(data=raw_importances,
                                 columns=feature_name)
    # order by average importance across folds
    sorted_indices = importance_df.mean(axis=0).sort_values(ascending=False).index
    sorted_importance_df = importance_df.loc[:, sorted_indices]
    # plot top-n
    PLOT_TOP_N = 50
    plot_cols = sorted_importance_df.columns[:PLOT_TOP_N]
    _, ax = plt.subplots(figsize=figsize)
    ax.grid()
    ax.set_xscale('log')
    ax.set_ylabel('Feature')
    ax.set_xlabel('Importance')
    sns.boxplot(data=sorted_importance_df[plot_cols],
                orient='h',
                ax=ax)
    plt.show()

def get_X(df_src):
    cols = [c for c in df_src.columns if c not in ['time_id', 'target', 'tick_size']]
    return df_src[cols]

class EnsembleModel:
    def __init__(self, models: List[lgb.Booster], weights: Optional[List[float]] = None):
        self.models = models
        self.weights = weights

        features = list(self.models[0].feature_name())

        for m in self.models[1:]:
            assert features == list(m.feature_name())

    def predict(self, x):
        predicted = np.zeros((len(x), len(self.models)))

        for i, m in enumerate(self.models):
            w = self.weights[i] if self.weights is not None else 1
            predicted[:, i] = w * m.predict(x)

        ttl = np.sum(self.weights) if self.weights is not None else len(self.models)
        return np.sum(predicted, axis=1) / ttl

    def feature_name(self) -> List[str]:
        return self.models[0].feature_name()

In [110]:
# add_results_from_light_gbm
def add_results_from_light_gbm(X_train_lgbm, y_train_lgbm, X_test_lgbm, y_test_lgbm, feature, lr, stock_ids):
    params = {
    'objective': 'regression',
    'verbose': 0,
    'metric': '',
    'reg_alpha': 5,
    'reg_lambda': 5,
    'min_data_in_leaf': 1000,
    'max_depth': -1,
    'num_leaves': 128,
    'colsample_bytree': 0.3,
    'learning_rate': lr
    }

    start_time = datetime.now()
    ds = lgb.Dataset(X_train_lgbm, y_train_lgbm, weight=1/np.power(y_train_lgbm, 2))

    ret = lgb.cv(params, ds, num_boost_round=8000,
                    feval=feval_RMSPE,
                    stratified=False,
                    return_cvbooster=True,
                    verbose_eval=20,
                    early_stopping_rounds=int(40*0.1/lr))

    # print(f"# overall RMSPE: {ret['RMSPE-mean'][-1]}")

    best_iteration = len(ret['RMSPE-mean'])

    # print('boosters length ', len(ret['cvbooster'].boosters))

    best_mae = None
    best_y_pred = None

    for i in range(len(ret['cvbooster'].boosters)):
        y_pred = ret['cvbooster'].boosters[i].predict(X_test_lgbm, num_iteration=best_iteration)
        # print('y_pred here ', y_pred)
        mae_value = mean_absolute_error(y_test_lgbm, y_pred)
        print('mae value ', mae_value)
        if best_mae == None:
            best_mae = mae_value

        if mae_value < best_mae:
            print('updating best mae value')
            best_mae = mae_value
            best_y_pred = y_pred

    time_taken = datetime.now() - start_time
    add_model_result('LightGBM', y_test_lgbm, best_y_pred, False, feature, time_taken, lr, EPOCHS, stock_ids)



In [111]:
# plot_importance(ret['cvbooster'], figsize=(10, 20))

## NN Training

In [112]:
# NN Training

NUM_WORKERS = 0 #4
MULTIPROCESSING_CONTEXT = 'spawn'

null_check_cols = [
    'book.log_return1.realized_volatility',
    'book_150.log_return1.realized_volatility',
    'book_300.log_return1.realized_volatility',
    'book_450.log_return1.realized_volatility',
    'trade.log_return.realized_volatility',
    'trade_150.log_return.realized_volatility',
    'trade_300.log_return.realized_volatility',
    'trade_450.log_return.realized_volatility'
]

def seed_everything(seed=0):
    random.seed(seed)
    os.environ['PYTHONHASHSEED'] = str(seed)
    np.random.seed(seed)
    torch.manual_seed(seed)
    torch.cuda.manual_seed(seed)
    torch.backends.cudnn.deterministic = True


def rmspe_metric(y_true, y_pred):
    rmspe = np.sqrt(np.mean(np.square((y_true - y_pred) / y_true)))
    return rmspe


def rmspe_loss(y_true, y_pred):
    rmspe = torch.sqrt(torch.mean(torch.square((y_true - y_pred) / y_true)))
    return rmspe


class RMSPE(Metric):
    def __init__(self):
        self._name = "rmspe"
        self._maximize = False

    def __call__(self, y_true, y_score):
        return np.sqrt(np.mean(np.square((y_true - y_score) / y_true)))

def RMSPELoss_Tabnet(y_pred, y_true):
    return torch.sqrt(torch.mean( ((y_true - y_pred) / y_true) ** 2 )).clone()


class AverageMeter:
    """Computes and stores the average and current value"""

    def __init__(self):
        self.val = 0
        self.avg = 0
        self.sum = 0
        self.count = 0
        self.reset()

    def reset(self):
        self.val = 0
        self.avg = 0
        self.sum = 0
        self.count = 0

    def update(self, val, n=1):
        self.val = val
        self.sum += val * n
        self.count += n
        self.avg = self.sum / self.count


class TabularDataset(Dataset):
    def __init__(self, x_num: np.ndarray, y: Optional[np.ndarray]):
        super().__init__()
        self.x_num = x_num
        self.y = y

    def __len__(self):
        return len(self.x_num)

    def __getitem__(self, idx):
        if self.y is None:
            return self.x_num[idx]
        else:
            return self.x_num[idx], self.y[idx]

class MLP(nn.Module):
    def __init__(self,
                 src_num_dim: int,
                 dropout: float = 0.0,
                 hidden: int = 50,
                 bn: bool = False):
        super().__init__()

        if bn:
            self.sequence = nn.Sequential(
                nn.Linear(src_num_dim, hidden),
                nn.Dropout(dropout),
                nn.BatchNorm1d(hidden),
                nn.ReLU(),
                nn.Linear(hidden, hidden),
                nn.Dropout(dropout),
                nn.BatchNorm1d(hidden),
                nn.ReLU(),
                nn.Linear(hidden, 1)
            )
        else:
            self.sequence = nn.Sequential(
                nn.Linear(src_num_dim, hidden),
                nn.Dropout(dropout),
                nn.ReLU(),
                nn.Linear(hidden, hidden),
                nn.Dropout(dropout),
                nn.ReLU(),
                nn.Linear(hidden, 1)
            )

    def forward(self, x_num):
        x = self.sequence(x_num)
        return torch.squeeze(x)


class CNN(nn.Module):
    def __init__(self,
                 num_features: int,
                 hidden_size: int,
                 emb_dim: int = 10,
                 dropout_cat: float = 0.2,
                 channel_1: int = 256,
                 channel_2: int = 512,
                 channel_3: int = 512,
                 dropout_top: float = 0.1,
                 dropout_mid: float = 0.3,
                 dropout_bottom: float = 0.2,
                 weight_norm: bool = True,
                 two_stage: bool = True,
                 celu: bool = True,
                 kernel1: int = 5,
                 leaky_relu: bool = False):
        super().__init__()

        num_targets = 1

        cha_1_reshape = int(hidden_size / channel_1)
        cha_po_1 = int(hidden_size / channel_1 / 2)
        cha_po_2 = int(hidden_size / channel_1 / 2 / 2) * channel_3

        self.cha_1 = channel_1
        self.cha_2 = channel_2
        self.cha_3 = channel_3
        self.cha_1_reshape = cha_1_reshape
        self.cha_po_1 = cha_po_1
        self.cha_po_2 = cha_po_2
        self.two_stage = two_stage

        self.expand = nn.Sequential(
            nn.BatchNorm1d(num_features),
            nn.Dropout(dropout_top),
            nn.utils.weight_norm(nn.Linear(num_features, hidden_size), dim=None),
            nn.CELU(0.06) if celu else nn.ReLU()
        )

        def _norm(layer, dim=None):
            return nn.utils.weight_norm(layer, dim=dim) if weight_norm else layer

        self.conv1 = nn.Sequential(
            nn.BatchNorm1d(channel_1),
            nn.Dropout(dropout_top),
            _norm(nn.Conv1d(channel_1, channel_2, kernel_size=kernel1, stride=1, padding=kernel1 // 2, bias=False)),
            nn.ReLU(),
            nn.AdaptiveAvgPool1d(output_size=cha_po_1),
            nn.BatchNorm1d(channel_2),
            nn.Dropout(dropout_top),
            _norm(nn.Conv1d(channel_2, channel_2, kernel_size=3, stride=1, padding=1, bias=True)),
            nn.ReLU()
        )

        if self.two_stage:
            self.conv2 = nn.Sequential(
                nn.BatchNorm1d(channel_2),
                nn.Dropout(dropout_mid),
                _norm(nn.Conv1d(channel_2, channel_2, kernel_size=3, stride=1, padding=1, bias=True)),
                nn.ReLU(),
                nn.BatchNorm1d(channel_2),
                nn.Dropout(dropout_bottom),
                _norm(nn.Conv1d(channel_2, channel_3, kernel_size=5, stride=1, padding=2, bias=True)),
                nn.ReLU()
            )

        self.max_po_c2 = nn.MaxPool1d(kernel_size=4, stride=2, padding=1)

        self.flt = nn.Flatten()

        if leaky_relu:
            self.dense = nn.Sequential(
                nn.BatchNorm1d(cha_po_2),
                nn.Dropout(dropout_bottom),
                _norm(nn.Linear(cha_po_2, num_targets), dim=0),
                nn.LeakyReLU()
            )
        else:
            self.dense = nn.Sequential(
                nn.BatchNorm1d(cha_po_2),
                nn.Dropout(dropout_bottom),
                _norm(nn.Linear(cha_po_2, num_targets), dim=0)
            )

    def forward(self, x_num):
        x = self.expand(x_num)
        x = x.reshape(x.shape[0], self.cha_1, self.cha_1_reshape)
        x = self.conv1(x)
        if self.two_stage:
            x = self.conv2(x) * x

        x = self.max_po_c2(x)
        x = self.flt(x)
        x = self.dense(x)

        return torch.squeeze(x)

# def preprocess_nn(
#         X: pd.DataFrame,
#         scaler: Optional[StandardScaler] = None,
#         scaler_type: str = 'standard',
#         n_pca: int = -1,
#         na_cols: bool = True):
#     if na_cols:
#         #for c in X.columns:
#         for c in null_check_cols:
#             if c in X.columns:
#                 X[f"{c}_isnull"] = X[c].isnull().astype(int)

#     cat_cols = [c for c in X.columns if c in ['time_id', 'stock_id']]
#     num_cols = [c for c in X.columns if c not in cat_cols]

#     X_num = X[num_cols].values.astype(np.float32)
#     X_cat = np.nan_to_num(X[cat_cols].values.astype(np.int32))

#     def _pca(X_num_):
#         if n_pca > 0:
#             pca = PCA(n_components=n_pca, random_state=0)
#             return pca.fit_transform(X_num)
#         return X_num

#     if scaler is None:
#         scaler = StandardScaler()
#         X_num = scaler.fit_transform(X_num)
#         X_num = np.nan_to_num(X_num, posinf=0, neginf=0)
#         return _pca(X_num), X_cat, cat_cols, scaler
#     else:
#         X_num = scaler.transform(X_num) #TODO: infでも大丈夫？
#         X_num = np.nan_to_num(X_num, posinf=0, neginf=0)
#         return _pca(X_num), X_cat, cat_cols


def train_epoch(data_loader: DataLoader,
                model: nn.Module,
                optimizer,
                scheduler,
                device,
                clip_grad: float = 1.5):
    model.train()
    losses = AverageMeter()
    step = 0

    # for x_num, y in tqdm(data_loader, position=0, leave=True, desc='Training'):
    for x_num, y in data_loader:
        batch_size = x_num.size(0)
        x_num = x_num.to(device, dtype=torch.float)
        y = y.to(device, dtype=torch.float)
        loss = rmspe_loss(y, model(x_num))
        losses.update(loss.detach().cpu().numpy(), batch_size)
        loss.backward()

        torch.nn.utils.clip_grad_norm_(model.parameters(), clip_grad)
        optimizer.step()
        optimizer.zero_grad()

        if scheduler is not None:
            scheduler.step()

        step += 1

    return losses.avg


def evaluate(data_loader: DataLoader, model, device):
    model.eval()

    losses = AverageMeter()

    final_targets = []
    final_outputs = []

    with torch.no_grad():
        # for x_num, y in tqdm(data_loader, position=0, leave=True, desc='Evaluating'):
        for x_num, y in data_loader:
            batch_size = x_num.size(0)
            x_num = x_num.to(device, dtype=torch.float)
            y = y.to(device, dtype=torch.float)

            with torch.no_grad():
                output = model(x_num)

            loss = rmspe_loss(y, output)
            losses.update(loss.detach().cpu().numpy(), batch_size)

            targets = y.detach().cpu().numpy()
            output = output.detach().cpu().numpy()

            final_targets.append(targets)
            final_outputs.append(output)

    final_targets = np.concatenate(final_targets)
    final_outputs = np.concatenate(final_outputs)

    try:
        metric = rmspe_metric(final_targets, final_outputs)
    except:
        metric = None

    return final_outputs, final_targets, losses.avg, metric

def predict_nn(X_df: pd.DataFrame,
               model: Union[List[MLP], MLP],
               device,
               ensemble_method='mean'):
    if not isinstance(model, list):
        model = [model]

    for m in model:
        m.eval()

    evaluation_dataset = TabularDataset(X_df.values, None)
    evaluation_data_loader = torch.utils.data.DataLoader(evaluation_dataset,
                                               batch_size=512,
                                               shuffle=False,
                                            #    multiprocessing_context=MULTIPROCESSING_CONTEXT,
                                               num_workers=NUM_WORKERS)

    final_outputs = []

    with torch.no_grad():
        # for x_num in tqdm(evaluation_data_loader, position=0, leave=True, desc='Evaluating'):
        for x_num in evaluation_data_loader:
            x_num = x_num.to(device, dtype=torch.float)

            outputs = []
            with torch.no_grad():
                for m in model:
                    output = m(x_num)
                    outputs.append(output.detach().cpu().numpy())

            if ensemble_method == 'median':
                pred = np.nanmedian(np.array(outputs), axis=0)
            else:
                pred = np.array(outputs).mean(axis=0)
            final_outputs.append(pred)

    final_outputs = np.concatenate(final_outputs)
    return final_outputs


def train_nn(
             X_train_df,
             y_train_df,
             X_val_df,
             y_val_df,
             device,
             emb_dim: int = 25,
             batch_size: int = 1024,
             model_type: str = 'mlp',
             mlp_dropout: float = 0.0,
             mlp_hidden: int = 64,
             mlp_bn: bool = False,
             cnn_hidden: int = 64,
             cnn_channel1: int = 32,
             cnn_channel2: int = 32,
             cnn_channel3: int = 32,
             cnn_kernel1: int = 5,
             cnn_celu: bool = False,
             cnn_weight_norm: bool = False,
             dropout_emb: bool = 0.0,
             lr: float = 1e-3,
             weight_decay: float = 0.0,
             model_path: str = 'fold_{}.pth',
             scaler_type: str = 'standard',
             output_dir: str = 'artifacts',
             scheduler_type: str = 'onecycle',
             optimizer_type: str = 'adam',
             max_lr: float = 0.01,
             epochs: int = 30,
             seed: int = 42,
             n_pca: int = -1,
             batch_double_freq: int = 50,
             cnn_dropout: float = 0.1,
             na_cols: bool = True,
             cnn_leaky_relu: bool = False,
             patience: int = 8,
             factor: float = 0.5):
    seed_everything(seed)

    os.makedirs(output_dir, exist_ok=True)

    best_losses = []
    best_predictions = []

    cur_batch = batch_size
    best_loss = 1e10
    best_prediction = None
    train_dataset = TabularDataset(X_train_df.values, y_train_df.values)
    valid_dataset = TabularDataset(X_val_df.values, y_val_df.values)
    train_loader = torch.utils.data.DataLoader(train_dataset, batch_size=cur_batch, shuffle=False,
                                                num_workers=NUM_WORKERS,
                                                # multiprocessing_context=MULTIPROCESSING_CONTEXT
                                                )
    valid_loader = torch.utils.data.DataLoader(valid_dataset, batch_size=cur_batch, shuffle=False,
                                            #    multiprocessing_context=MULTIPROCESSING_CONTEXT,
                                                num_workers=NUM_WORKERS)

    if model_type == 'mlp':
        model = MLP(X_train_df.shape[1],
                    dropout=mlp_dropout,
                    hidden=mlp_hidden,
                    bn=mlp_bn)
    elif model_type == 'cnn':
        model = CNN(X_train_df.shape[1],
                    hidden_size=cnn_hidden,
                    emb_dim=emb_dim,
                    dropout_cat=dropout_emb,
                    channel_1=cnn_channel1,
                    channel_2=cnn_channel2,
                    channel_3=cnn_channel3,
                    two_stage=False,
                    kernel1=cnn_kernel1,
                    celu=cnn_celu,
                    dropout_top=cnn_dropout,
                    dropout_mid=cnn_dropout,
                    dropout_bottom=cnn_dropout,
                    weight_norm=cnn_weight_norm,
                    leaky_relu=cnn_leaky_relu)
    else:
        raise NotImplementedError()
    model = model.to(device)

    if optimizer_type == 'adamw':
        opt = torch.optim.AdamW(model.parameters(), lr=lr, weight_decay=weight_decay)
    elif optimizer_type == 'adam':
        opt = torch.optim.Adam(model.parameters(), lr=lr, weight_decay=weight_decay)
    else:
        raise NotImplementedError()

    scheduler = epoch_scheduler = None
    if scheduler_type == 'onecycle':
        scheduler = torch.optim.lr_scheduler.OneCycleLR(optimizer=opt, pct_start=0.1, div_factor=1e3,
                                                        max_lr=max_lr, epochs=epochs,
                                                        steps_per_epoch=len(train_loader))
    elif scheduler_type == 'reduce':
        epoch_scheduler = torch.optim.lr_scheduler.ReduceLROnPlateau(optimizer=opt,
                                                                        mode='min',
                                                                        min_lr=1e-7,
                                                                        patience=patience,
                                                                        verbose=False,
                                                                        factor=factor)

    for epoch in range(epochs):
        if epoch > 0 and epoch % batch_double_freq == 0:
            cur_batch = cur_batch * 2
            # print(f'batch: {cur_batch}')
            train_loader = torch.utils.data.DataLoader(train_dataset,
                                                        batch_size=cur_batch,
                                                        shuffle=False,
                                                        # multiprocessing_context=MULTIPROCESSING_CONTEXT,
                                                        num_workers=NUM_WORKERS)

        train_loss = train_epoch(train_loader, model, opt, scheduler, device)
        predictions, valid_targets, valid_loss, rmspe = evaluate(valid_loader, model, device=device)
        # print(f"epoch {epoch}, train loss: {train_loss:.3f}, valid rmspe: {rmspe:.3f}")

        if epoch_scheduler is not None:
            epoch_scheduler.step(rmspe)

        if rmspe < best_loss:
            # print(f'new best:{rmspe}')
            best_loss = rmspe
            best_prediction = predictions
            model_save_path = DATA_DIR + "/" + os.path.join(output_dir, model_path.format(0))
            torch.save(model, model_save_path)

    best_predictions.append(best_prediction)
    best_losses.append(best_loss)
    # del model, train_dataset, valid_dataset, train_loader, valid_loader, X_tr, X_va, X_tr_cat, X_va_cat, y_tr, y_va, opt
    del train_dataset, valid_dataset, train_loader, valid_loader, opt
    if scheduler is not None:
        del scheduler
    gc.collect()
    # , scaler
    return model, best_losses, best_predictions


In [113]:
# get_device_name
def get_device_name():
    if torch.backends.mps.is_available():
        return "mps"

    if torch.cuda.is_available():
        return "cuda"

    return "cpu"
device = torch.device(get_device_name())
# device = "cpu"
print('device', device)

# del df, df_train
gc.collect()

def get_top_n_models(models, scores, top_n):
    if len(models) <= top_n:
        print('number of models are less than top_n. all models will be used')
        return models
    sorted_ = [(y, x) for y, x in sorted(zip(scores, models), key=lambda pair: pair[0])]
    print(f'scores(sorted): {[y for y, _ in sorted_]}')
    return [x for _, x in sorted_][:top_n]

device mps


In [114]:
# add_results_for_mlp
def add_results_for_mlp(X_train_mlp, y_train_mlp, X_val_mlp, y_val_mlp, X_test_mlp, y_test_mlp, feature, 
                        epochs, learning_rate, stock_ids: list[int]):
    start_time = datetime.now()
    model_mlp, nn_losses, nn_preds = train_nn(
                                            X_train_mlp,
                                            y_train_mlp,
                                            X_val_mlp,
                                            y_val_mlp,
                                            device=device,
                                            batch_size=512,
                                            mlp_bn=True,
                                            mlp_hidden=256,
                                            mlp_dropout=0.0,
                                            emb_dim=30,
                                            epochs=epochs,
                                            lr=learning_rate,
                                            max_lr=0.0055,
                                            weight_decay=1e-7,
                                            model_path='mlp_fold_{}' + f"_seed{SEED}.pth",
                                            seed=0)

    model_mlp_preds = predict_nn(X_test_mlp, model_mlp, device, ensemble_method=ENSEMBLE_METHOD)
    time_taken = datetime.now() - start_time
    add_model_result('MLP', y_test_mlp, model_mlp_preds, False, feature, time_taken, 
                     learning_rate, epochs, stock_ids)


In [115]:
# add_results_for_cnn
# lr = 0.00038
def add_results_for_cnn(X_train_cnn, y_train_cnn, X_val_cnn, y_val_cnn, X_test_cnn, y_test_cnn, feature,
                        epochs, learning_rate, stock_ids: list[int]):
    start_time = datetime.now()
    model_cnn, nn_losses, nn_preds = train_nn(
                                            X_train_cnn,
                                            y_train_cnn,
                                            X_val_cnn,
                                            y_val_cnn,
                                            device=device,
                                            cnn_hidden=8*128,
                                            batch_size=1280,
                                            model_type='cnn',
                                            emb_dim=30,
                                            epochs=epochs,
                                            cnn_channel1=128,
                                            cnn_channel2=3*128,
                                            cnn_channel3=3*128,
                                            lr=learning_rate, #0.0011,
                                            max_lr=0.0013,
                                            weight_decay=6.5e-6,
                                            optimizer_type='adam',
                                            scheduler_type='reduce',
                                            model_path='cnn_fold_{}' + f"_seed{SEED}.pth",
                                            seed=0,
                                            cnn_dropout=0.0,
                                            cnn_weight_norm=False, # Note: True
                                            cnn_leaky_relu=False,
                                            patience=8,
                                            factor=0.3)

    model_cnn_preds = predict_nn(X_test_cnn, model_cnn, device, ensemble_method=ENSEMBLE_METHOD)
    time_taken = datetime.now() - start_time
    add_model_result('CNN', y_test_cnn, model_cnn_preds, False, feature, time_taken, 
                     learning_rate, epochs, stock_ids)



In [116]:
# create_timeseries_data
def create_timeseries_data(df):
    df_ts = TimeSeries.from_dataframe(df)
    scaler = Scaler()
    df_ts = scaler.fit_transform(df_ts).astype(np.float32)
    # print('Length of Timeseries ', len(df_ts))
    return df_ts, scaler

In [117]:
# add_results_for_TCN
def add_results_for_TCN(X_train_ts_tcn, y_train_ts_tcn, X_val_ts_tcn, y_val_ts_tcn, X_test_ts_tcn, y_test_ts_tcn,
    feature, epochs, learning_rate, stock_ids: list[int]):
    time_start = datetime.now()
    model_tcn = TCNModel(
        input_chunk_length=72,
        output_chunk_length=1,
        n_epochs=epochs, #500
        dropout=0.1,
        dilation_base=2,
        weight_norm=False,
        kernel_size=3,
        num_filters=3,
        random_state=0,
        optimizer_kwargs = {'lr': learning_rate}
    )

    model_tcn.fit(
        series=y_train_ts_tcn,
        past_covariates=X_train_ts_tcn,
        val_series=y_val_ts_tcn,
        val_past_covariates=X_val_ts_tcn,
        verbose=False
    )

    predictions_tcn = model_tcn.historical_forecasts(
        series=y_test_ts_tcn,
        past_covariates=X_test_ts_tcn,
        forecast_horizon=1,
        retrain=False,
        verbose=False,
    )

    # predictions_tcn = model_tcn.predict(n = 36,
    #                                 past_covariates=X_test_ts_tcn,
    #                                 verbose=False)

    time_taken = datetime.now() - time_start
    add_model_result('TCN', y_test_ts_tcn, predictions_tcn, True, feature, time_taken, 
                     learning_rate, epochs, stock_ids)


In [118]:
# plot_tcn_predictions
def plot_tcn_predictions(y_test_ts_tcn, backtest_tcn):
    y_test_ts_tcn.plot(label="actual")
    backtest_tcn.plot(label="backtest (H=6)")
    plt.legend()

In [119]:
# add_results_for_lstm
def add_results_for_lstm(X_train_ts_lstm, y_train_ts_lstm, X_val_ts_lstm, y_val_ts_lstm, X_test_ts_lstm, 
                         y_test_ts_lstm, 
                        feature, epochs, learning_rate, stock_ids: list[int]):
    start_time = datetime.now()
    model_lstm = RNNModel(
        model="LSTM",
        hidden_dim=20,
        n_rnn_layers=2,
        dropout=0.2,
        batch_size=16,
        n_epochs=epochs,
        optimizer_kwargs={"lr": learning_rate},
        random_state=0,
        training_length=72,
        input_chunk_length=72,
        output_chunk_length=1,
        likelihood=GaussianLikelihood(),
    )

    model_lstm.fit(
        series=y_train_ts_lstm,
        future_covariates=X_train_ts_lstm,
        val_series=y_val_ts_lstm,
        val_future_covariates=X_val_ts_lstm,
        verbose=False,
    )

    predictions_lstm = model_lstm.historical_forecasts(
        series=y_test_ts_lstm,
        future_covariates=X_test_ts_lstm,
        forecast_horizon=1,
        retrain=False,
        verbose=False,
    )

    # predictions_lstm = model_lstm.predict(n = 36, past_covariates = X_test_ts_lstm, verbose=True)

    time_taken = datetime.now() - start_time
    add_model_result('LSTM', y_test_ts_lstm, predictions_lstm, True, feature, time_taken, 
                     learning_rate, epochs, stock_ids)

In [120]:
# plot_lstm_predictions
def plot_lstm_predictions(y_test_ts_lstm, backtest_lstm):
    y_test_ts_lstm.plot(label="actual")
    backtest_lstm.plot(label="backtest (H=6)")
    plt.legend()

In [121]:
# add_results_for_transformer
def add_results_for_transformer(X_train_ts_trans, y_train_ts_trans, X_val_ts_trans, y_val_ts_trans, X_test_ts_trans, 
    y_test_ts_trans, feature, epochs, learning_rate, stock_ids: list[int]):
    start_time = datetime.now()
    model_transformer = TransformerModel(
        input_chunk_length=72,
        output_chunk_length=1,
        batch_size=32,
        n_epochs=epochs,
        model_name="stock_transformer_"+feature,
        nr_epochs_val_period=10,
        d_model=16,
        nhead=8,
        num_encoder_layers=2,
        num_decoder_layers=2,
        dim_feedforward=128,
        dropout=0.1,
        activation="relu",
        random_state=0,
        save_checkpoints=True,
        force_reset=True,
        optimizer_kwargs = {'lr': learning_rate}
    )

    model_transformer.fit(
        series=y_train_ts_trans,
        past_covariates=X_train_ts_trans,
        val_series=y_val_ts_trans,
        val_past_covariates=X_val_ts_trans,
        verbose=False,
    )

    predictions_transformer = model_transformer.historical_forecasts(
        series=y_test_ts_trans,
        past_covariates=X_test_ts_trans,
        forecast_horizon=1,
        retrain=False,
        verbose=False,
    )

    # predictions_transformer = model_transformer.predict(n=36, series=y_test_ts_trans, past_covariates=X_test_ts_trans)

    time_taken = datetime.now() - start_time
    add_model_result('Transformer', y_test_ts_trans, predictions_transformer, True, feature, 
                     time_taken, learning_rate, epochs, stock_ids)

In [122]:
epochs = [3]

In [123]:
def add_results_for_mlp_with_hyperparameter_tuning(X_train, y_train, X_val, y_val, X_test, y_test, 
                                                   feature, stock_ids: list[int]):
    # call this in parallel
    # learning_rates = [0.002, 0.001, 0.01]
    learning_rates = [0.002]
    for lr in learning_rates:
        for epoch in epochs:
            print('add_results_for_mlp lr ', lr, ', epoch ', epoch)
            add_results_for_mlp(X_train, y_train, X_val, y_val, X_test, y_test, feature, epoch, lr, stock_ids)
              

In [124]:
def add_results_for_cnn_with_hyperparameter_tuning(X_train, y_train, X_val, y_val, X_test, y_test, 
                                                   feature, stock_ids: list[int]):
    #0.00038
    # learning_rates = [0.00038, 0.001, 0.0001]
    learning_rates = [0.00038]
    for lr in learning_rates:
        for epoch in epochs:
            print('add_results_for_cnn lr ', lr, ', epoch ', epoch)
            add_results_for_cnn(X_train, y_train, X_val, y_val, X_test, y_test, feature, epoch, lr, stock_ids)

In [125]:
def add_results_for_TCN_with_hyperparameter_tuning(X_train, y_train, X_val, y_val, X_test, y_test, 
                                                   feature, stock_ids: list[int]):
    # learning_rates = [0.002, 0.001, 0.01]
    learning_rates = [0.01]
    for lr in learning_rates:
        for epoch in epochs:
            print('add_results_for_TCN lr ', lr, ', epoch ', epoch)
            add_results_for_TCN(X_train, y_train, X_val, y_val, X_test, y_test, feature, epoch, lr, stock_ids)

In [126]:
def add_results_for_lstm_with_hyperparameter_tuning(X_train, y_train, X_val, y_val, X_test, y_test, 
                                                    feature, stock_ids: list[int]):
    # learning_rates = [0.002, 0.001, 0.01]
    learning_rates = [0.01]
    for lr in learning_rates:
        for epoch in epochs:
            print('add_results_for_lstm lr ', lr, ', epoch ', epoch)
            add_results_for_lstm(X_train, y_train, X_val, y_val, X_test, y_test, feature, epoch, lr, 
                                 stock_ids)


In [127]:
def add_results_for_transformer_with_hyperparameter_tuning(X_train, y_train, X_val, y_val, X_test, y_test, 
                                                           feature, stock_ids: list[int]):
    # learning_rates = [0.002, 0.001, 0.01]
    learning_rates = [0.01]
    for lr in learning_rates:
        for epoch in epochs:
            print('add_results_for_transformer lr ', lr, ', epoch ', epoch)
            add_results_for_transformer(X_train, y_train, X_val, y_val, X_test, y_test, feature, epoch, 
                                        lr, stock_ids)

In [128]:
# perform_experiments_multivariate
def perform_experiments_multivariate(df_experiment, feature: str, stock_ids: list[int], stock_ids_in_test: list[int]):
    print('perform_experiments_multivariate ', feature)
    df_train, df_validation, df_test = split_df_into_train_val_test(df_experiment, 0.8, 0.5)

    if stock_ids_in_test:
        df_test = df_test[df_test['stock_id'].isin(stock_ids_in_test)].reset_index(drop=True)

    print('Unique stock ids in test ', df_test['stock_id'].unique())

    print('df_train ', df_train.shape)
    print('df_val ', df_validation.shape)
    print('df_test ', df_test.shape)

    # prepare train, validation and test data
    X_train = get_X(df_train)
    X_val = get_X(df_validation)
    X_test = get_X(df_test)

    # print('xtrain')
    # display(X_train.head())
    # sdf

    y_train = df_train['target']
    y_val = df_validation['target']
    y_test = df_test['target']

    X_train_ts, X_train_ts_scaler = create_timeseries_data(X_train)
    X_val_ts, X_val_ts_scaler = create_timeseries_data(X_val)
    X_test_ts, X_test_ts_scaler = create_timeseries_data(X_test)

    y_train_ts, y_train_ts_scaler = create_timeseries_data(y_train.to_frame())
    y_val_ts, y_val_ts_scaler = create_timeseries_data(y_val.to_frame())
    y_test_ts, y_test_ts_scaler = create_timeseries_data(y_test.to_frame())

    # X_ts = X_train_ts.append(X_val_ts)
    # y_ts = y_train_ts.append(y_val_ts)

    add_results_from_light_gbm(X_train, y_train, X_val, y_val, feature, 0.3, stock_ids)

    add_results_for_mlp_with_hyperparameter_tuning(X_train, y_train, X_val, y_val, X_test, y_test, feature, stock_ids)
    add_results_for_cnn_with_hyperparameter_tuning(X_train, y_train, X_val, y_val, X_test, y_test, feature, stock_ids)
    add_results_for_TCN_with_hyperparameter_tuning(X_train_ts, y_train_ts, X_val_ts, y_val_ts, X_test_ts, y_test_ts, feature, stock_ids)
    add_results_for_lstm_with_hyperparameter_tuning(X_train_ts, y_train_ts, X_val_ts, y_val_ts, X_test_ts, y_test_ts, feature, stock_ids)
    # add_results_for_transformer_with_hyperparameter_tuning(X_train_ts, y_train_ts, X_val_ts, y_val_ts, X_test_ts, y_test_ts, feature, stock_ids)

    return get_model_results_df()


In [129]:
def get_all_train_data():
    train = pd.read_csv(os.path.join(DATA_DIR, 'optiver-realized-volatility-prediction', 'train_time_id_ordered.csv'))
    return train

def get_unique_stock_ids():
    train = get_all_train_data()
    return list(train['stock_id'].unique())

def get_train_data(stock_ids_to_include):
    train = get_all_train_data()
    # stock_ids = set(train['stock_id'])
    print('Train.shape all ', train.shape)
    # print('stock_ids ', len(stock_ids))

    train = train[train['stock_id'].isin(stock_ids_to_include)]
    train = train.reset_index(drop=True)
    print('Train.shape sample ', train.shape)
    # stock_ids = set(train['stock_id'])
    # print('stock_ids ', stock_ids)

    return train

In [130]:
# Nearest neighbor features
def perform_multivariate_experiments_with_nearest_neighbor_features(
        stock_data,
        n_neighbors,
        use_price_nn_features, 
        use_volume_nn_features, 
        use_size_nn_features, 
        use_random_nn_features,
        feature,
        stock_ids: list[int],
        stock_ids_in_test: list[int]):
    print('nearest neighbor feature ', feature)
    with_nn_df = make_features(base=stock_data,
                            block=DataBlock.TRAIN,
                            add_spread_features=True, 
                            add_statistics_features=True,
                            add_book_time_features=True,
                            add_trade_time_features=True)
    with_nn_df = make_features_tick_size(with_nn_df, DataBlock.TRAIN)
    with_nn_df = add_tau_features(with_nn_df)
    time_id_neighbors, stock_id_neighbors = build_nearest_neighbors(with_nn_df, 
                                                                    n_neighbors=n_neighbors,
                                                                    use_price_nn_features=use_price_nn_features, 
                                                                    use_volume_nn_features=use_volume_nn_features, 
                                                                    use_size_nn_features=use_size_nn_features, 
                                                                    use_random_nn_features=use_random_nn_features)
    with_nn_df = make_nearest_neighbor_feature(df_nn=with_nn_df,
                                            time_id_neighbors=time_id_neighbors,
                                            stock_id_neighbors=stock_id_neighbors,
                                            use_price_nn_features=True)
    
    print(f'{feature} columns ', len(with_nn_df.columns), ', ', with_nn_df.columns)

    perform_experiments_multivariate(with_nn_df, feature, stock_ids, stock_ids_in_test)

In [131]:
def get_number_of_neighbors(num_stock_ids):
    if num_stock_ids > 80:
        return 80
    
    return num_stock_ids

def perform_multivariate_experiments_for_stock(stock_ids: list[int], stock_ids_in_test: list[int]):
    # print('perform_multivariate_experiments_for_stock ', perform_multivariate_experiments_for_stock)
    stock_data = get_train_data(stock_ids)
    # Feature lag
    df = make_rv_lags(stock_data, 9*5)
    print('rv_lags columns ', len(df.columns), ', ', df.columns)
    perform_experiments_multivariate(df, 'rv_lags', stock_ids, stock_ids_in_test)
    # Feature set: log_return
    
    df = make_features(base=stock_data,
                            block=DataBlock.TRAIN,
                            add_spread_features=False, 
                            add_statistics_features=False,
                            add_book_time_features=False,
                            add_trade_time_features=False)
    print('log return columns ', len(df.columns), ', ', df.columns)
    perform_experiments_multivariate(df, 'log_return', stock_ids, stock_ids_in_test)
    
    # Feature set: log_return + spread features
    df = make_features(base=stock_data,
                            block=DataBlock.TRAIN,
                            add_spread_features=True, 
                            add_statistics_features=False,
                            add_book_time_features=False,
                            add_trade_time_features=False)
    print('log_return_and_spread_features columns ', len(df.columns), ', ', df.columns)
    perform_experiments_multivariate(df, 'log_return_and_spread_features', stock_ids, stock_ids_in_test)

    # Feature set log_return, spread, statistics
    df = make_features(base=stock_data,
                           block=DataBlock.TRAIN,
                           add_spread_features=True, 
                           add_statistics_features=True,
                           add_book_time_features=False,
                           add_trade_time_features=False)

    print('log_return_spread_and_statistics_features columns ', len(df.columns), ', ', df.columns)
    perform_experiments_multivariate(df, 'log_return_spread_and_statistics_features', stock_ids, stock_ids_in_test)

    if len(stock_ids) > 1:
        print('Nearest neighbor features ')
        perform_multivariate_experiments_with_nearest_neighbor_features(
            stock_data=stock_data,
            n_neighbors=get_number_of_neighbors(len(stock_ids)),
            use_price_nn_features=True, 
            use_volume_nn_features=False, 
            use_size_nn_features=False, 
            use_random_nn_features=False,
            feature='price_nn_features',
            stock_ids=stock_ids,
            stock_ids_in_test=stock_ids_in_test)
        
        perform_multivariate_experiments_with_nearest_neighbor_features(
            stock_data=stock_data, 
            n_neighbors=get_number_of_neighbors(len(stock_ids)),
            use_price_nn_features=True, 
            use_volume_nn_features=True, 
            use_size_nn_features=False, 
            use_random_nn_features=False,
            feature='volume_nn_features',
            stock_ids=stock_ids,
            stock_ids_in_test=stock_ids_in_test)
        
        perform_multivariate_experiments_with_nearest_neighbor_features( 
            stock_data=stock_data,
            n_neighbors=get_number_of_neighbors(len(stock_ids)),
            use_price_nn_features=True, 
            use_volume_nn_features=False, 
            use_size_nn_features=True, 
            use_random_nn_features=False,
            feature='size_nn_features',
            stock_ids=stock_ids,
            stock_ids_in_test=stock_ids_in_test)
        
        perform_multivariate_experiments_with_nearest_neighbor_features(
            stock_data=stock_data,
            n_neighbors=get_number_of_neighbors(len(stock_ids)),
            use_price_nn_features=True, 
            use_volume_nn_features=False, 
            use_size_nn_features=False, 
            use_random_nn_features=True,
            feature='random_nn_features',
            stock_ids=stock_ids,
            stock_ids_in_test=stock_ids_in_test)
        
        perform_multivariate_experiments_with_nearest_neighbor_features(
            stock_data=stock_data,
            n_neighbors=get_number_of_neighbors(len(stock_ids)),
            use_price_nn_features=True, 
            use_volume_nn_features=True, 
            use_size_nn_features=True, 
            use_random_nn_features=True,
            feature='all_nn_features',
            stock_ids=stock_ids,
            stock_ids_in_test=stock_ids_in_test)



In [132]:
get_all_train_data().head()

,stock_id,time_id,target,time_id_order
0,8,4294,0.005047,0
1,47,4294,0.001244,0
2,105,4294,0.002353,0
3,69,4294,0.001616,0
4,111,4294,0.001917,0


In [133]:
all_stock_ids = sorted(get_unique_stock_ids())

In [134]:
len(all_stock_ids)

112

In [135]:
# stock_data = get_train_data([0])
# df = make_features(base=stock_data,
#                         block=DataBlock.TRAIN,
#                         add_spread_features=False, 
#                         add_statistics_features=False,
#                         add_book_time_features=False,
#                         add_trade_time_features=False)
# print('df sh ', df.shape)
# df[['target','lag_1_rv']].head()


In [136]:
def get_top_n_results(n: int):
    df_res = get_model_results_df()
    df_res = df_res.sort_values(by=["rmse"], ascending=True)
    return df_res.head(n)

In [137]:

def perform_experiments(stock_ids_test, number_of_stocks):
    for i in number_of_stocks:
        print('Number of stocks processing ', i)
        stock_ids = all_stock_ids[0:i]
        # stock_ids = [0,1] a
        print('stock_ids = ', stock_ids)
        print('len(stock_ids) = ', len(stock_ids))
        perform_multivariate_experiments_for_stock(stock_ids, stock_ids_test)
        # display(get_top_n_results(3))

In [138]:
number_of_stocks = [112]
stock_ids_test = [0]
perform_experiments(stock_ids_test, number_of_stocks)

Number of stocks processing  112
stock_ids =  [0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 13, 14, 15, 16, 17, 18, 19, 20, 21, 22, 23, 26, 27, 28, 29, 30, 31, 32, 33, 34, 35, 36, 37, 38, 39, 40, 41, 42, 43, 44, 46, 47, 48, 50, 51, 52, 53, 55, 56, 58, 59, 60, 61, 62, 63, 64, 66, 67, 68, 69, 70, 72, 73, 74, 75, 76, 77, 78, 80, 81, 82, 83, 84, 85, 86, 87, 88, 89, 90, 93, 94, 95, 96, 97, 98, 99, 100, 101, 102, 103, 104, 105, 107, 108, 109, 110, 111, 112, 113, 114, 115, 116, 118, 119, 120, 122, 123, 124, 125, 126]
len(stock_ids) =  112
Train.shape all  (428932, 4)
Train.shape sample  (428932, 4)


rv_lags columns  49 ,  Index(['stock_id', 'time_id', 'target', 'time_id_order', 'lag_1_rv',
       'lag_2_rv', 'lag_3_rv', 'lag_4_rv', 'lag_5_rv', 'lag_6_rv', 'lag_7_rv',
       'lag_8_rv', 'lag_9_rv', 'lag_10_rv', 'lag_11_rv', 'lag_12_rv',
       'lag_13_rv', 'lag_14_rv', 'lag_15_rv', 'lag_16_rv', 'lag_17_rv',
       'lag_18_rv', 'lag_19_rv', 'lag_20_rv', 'lag_21_rv', 'lag_22_rv',
       'lag_23_rv', 'lag_24_rv', 'lag_25_rv', 'lag_26_rv', 'lag_27_rv',
       'lag_28_rv', 'lag_29_rv', 'lag_30_rv', 'lag_31_rv', 'lag_32_rv',
       'lag_33_rv', 'lag_34_rv', 'lag_35_rv', 'lag_36_rv', 'lag_37_rv',
       'lag_38_rv', 'lag_39_rv', 'lag_40_rv', 'lag_41_rv', 'lag_42_rv',
       'lag_43_rv', 'lag_44_rv', 'lag_45_rv'],
      dtype='object')
perform_experiments_multivariate  rv_lags
Unique stock ids in test  [0]
df_train  (343095, 49)
df_val  (42896, 49)
df_test  (383, 49)
[LightGBM] [Warning] Auto-choosing col-wise multi-threading, the overhead of testing was 0.021922 seconds.
You can set `forc

ValueError: Could not convert integer index to a pd.RangeIndex. Found non-unique step sizes/frequencies: {1, 2}. If any of those is the actual frequency, try passing it with fill_missing_dates=True and freq=your_frequency.

In [ ]:
# df_res = pd.read_csv('./data/model_results copy 3.csv')
# df_res

In [ ]:
df_res = get_model_results_df()
df_res

,model_name,mse,rmse,mae,rmspe,mape,added_date,feature,time_taken,learning_rate,epochs,stock_ids,stock_ids_count
0,MLP,3.845433,1.960978,1.957235,655.996614,600.710078,2023-10-02 12:40:38.858850,rv_lags,1.505546,0.00200,3,"[0, 1]",2
1,CNN,0.026277,0.162101,0.161746,54.656174,49.872181,2023-10-02 12:40:42.487133,rv_lags,3.613407,0.00038,3,"[0, 1]",2
2,TCN,0.019082,0.138137,0.102531,NaN,NaN,2023-10-02 12:41:08.124616,rv_lags,25.623504,0.01000,3,"[0, 1]",2
3,LSTM,0.029252,0.171031,0.130814,NaN,NaN,2023-10-02 12:41:38.374870,rv_lags,30.234473,0.01000,3,"[0, 1]",2
4,LightGBM,0.000002,0.001508,0.000878,0.268784,0.211913,2023-10-02 12:41:49.426212,log_return,0.475582,0.30000,10,"[0, 1]",2
5,MLP,0.002397,0.048964,0.048361,16.202856,14.749783,2023-10-02 12:41:50.240940,log_return,0.805806,0.00200,3,"[0, 1]",2
6,CNN,0.181218,0.425697,0.423900,143.417437,130.694040,2023-10-02 12:41:53.347976,log_return,3.104570,0.00038,3,"[0, 1]",2
7,TCN,0.018386,0.135596,0.101200,NaN,NaN,2023-10-02 12:42:19.007667,log_return,25.649386,0.01000,3,"[0, 1]",2
8,LSTM,0.046352,0.215294,0.182150,NaN,NaN,2023-10-02 12:42:48.941010,log_return,29.922036,0.01000,3,"[0, 1]",2
9,MLP,0.001174,0.034259,0.028424,11.522947,8.779802,2023-10-02 12:42:55.486987,log_return_and_spread_features,1.000329,0.00200,3,"[0, 1]",2


In [ ]:
# df_res = get_model_results_df()
df_res.sort_values(by=["mae"], ascending=True)

,model_name,mse,rmse,mae,rmspe,mape,added_date,feature,time_taken,learning_rate,epochs,stock_ids,stock_ids_count
37,LightGBM,0.000002,0.001439,0.000860,0.267869,0.210950,2023-10-02 12:50:35.952329,all_nn_features,1.564156,0.30000,10,"[0, 1]",2
27,LightGBM,0.000002,0.001461,0.000861,0.266591,0.208594,2023-10-02 12:47:53.316452,size_nn_features,0.809415,0.30000,10,"[0, 1]",2
32,LightGBM,0.000002,0.001450,0.000866,0.265938,0.211548,2023-10-02 12:49:14.342515,random_nn_features,2.069210,0.30000,10,"[0, 1]",2
22,LightGBM,0.000002,0.001443,0.000868,0.267282,0.211739,2023-10-02 12:46:36.824923,volume_nn_features,0.837450,0.30000,10,"[0, 1]",2
4,LightGBM,0.000002,0.001508,0.000878,0.268784,0.211913,2023-10-02 12:41:49.426212,log_return,0.475582,0.30000,10,"[0, 1]",2
13,LightGBM,0.000002,0.001477,0.000882,0.273266,0.216330,2023-10-02 12:43:59.487387,log_return_spread_and_statistics_features,0.539165,0.30000,10,"[0, 1]",2
28,MLP,0.000611,0.024715,0.019282,8.455343,5.806311,2023-10-02 12:47:54.243295,size_nn_features,0.891499,0.00200,3,"[0, 1]",2
18,MLP,0.000634,0.025170,0.020812,8.628183,6.617198,2023-10-02 12:45:20.568444,price_nn_features,1.572219,0.00200,3,"[0, 1]",2
9,MLP,0.001174,0.034259,0.028424,11.522947,8.779802,2023-10-02 12:42:55.486987,log_return_and_spread_features,1.000329,0.00200,3,"[0, 1]",2
10,CNN,0.001220,0.034935,0.028718,10.412834,8.222080,2023-10-02 12:42:58.689550,log_return_and_spread_features,3.195306,0.00038,3,"[0, 1]",2


In [ ]:
import uuid
print(uuid.uuid4())

47a73215-e047-4398-8614-966ca6073590


In [ ]:
output_file = f'./results/{uuid.uuid4()}.csv'
get_model_results_df().to_csv(output_file, index=False)

In [ ]:
df_res.sort_values(by=["rmspe"], ascending=True).head()

,model_name,mse,rmse,mae,rmspe,mape,added_date,feature,time_taken,learning_rate,epochs,stock_ids,stock_ids_count
32,LightGBM,0.000002,0.001450,0.000866,0.265938,0.211548,2023-10-02 12:49:14.342515,random_nn_features,2.069210,0.3,10,"[0, 1]",2
27,LightGBM,0.000002,0.001461,0.000861,0.266591,0.208594,2023-10-02 12:47:53.316452,size_nn_features,0.809415,0.3,10,"[0, 1]",2
22,LightGBM,0.000002,0.001443,0.000868,0.267282,0.211739,2023-10-02 12:46:36.824923,volume_nn_features,0.837450,0.3,10,"[0, 1]",2
37,LightGBM,0.000002,0.001439,0.000860,0.267869,0.210950,2023-10-02 12:50:35.952329,all_nn_features,1.564156,0.3,10,"[0, 1]",2
4,LightGBM,0.000002,0.001508,0.000878,0.268784,0.211913,2023-10-02 12:41:49.426212,log_return,0.475582,0.3,10,"[0, 1]",2


In [ ]:
df_res.sort_values(by=["rmspe"], ascending=True).head(20)

,model_name,mse,rmse,mae,rmspe,mape,added_date,feature,time_taken,learning_rate,epochs,stock_ids,stock_ids_count
32,LightGBM,0.000002,0.001450,0.000866,0.265938,0.211548,2023-10-02 12:49:14.342515,random_nn_features,2.069210,0.30000,10,"[0, 1]",2
27,LightGBM,0.000002,0.001461,0.000861,0.266591,0.208594,2023-10-02 12:47:53.316452,size_nn_features,0.809415,0.30000,10,"[0, 1]",2
22,LightGBM,0.000002,0.001443,0.000868,0.267282,0.211739,2023-10-02 12:46:36.824923,volume_nn_features,0.837450,0.30000,10,"[0, 1]",2
37,LightGBM,0.000002,0.001439,0.000860,0.267869,0.210950,2023-10-02 12:50:35.952329,all_nn_features,1.564156,0.30000,10,"[0, 1]",2
4,LightGBM,0.000002,0.001508,0.000878,0.268784,0.211913,2023-10-02 12:41:49.426212,log_return,0.475582,0.30000,10,"[0, 1]",2
13,LightGBM,0.000002,0.001477,0.000882,0.273266,0.216330,2023-10-02 12:43:59.487387,log_return_spread_and_statistics_features,0.539165,0.30000,10,"[0, 1]",2
28,MLP,0.000611,0.024715,0.019282,8.455343,5.806311,2023-10-02 12:47:54.243295,size_nn_features,0.891499,0.00200,3,"[0, 1]",2
18,MLP,0.000634,0.025170,0.020812,8.628183,6.617198,2023-10-02 12:45:20.568444,price_nn_features,1.572219,0.00200,3,"[0, 1]",2
10,CNN,0.001220,0.034935,0.028718,10.412834,8.222080,2023-10-02 12:42:58.689550,log_return_and_spread_features,3.195306,0.00038,3,"[0, 1]",2
9,MLP,0.001174,0.034259,0.028424,11.522947,8.779802,2023-10-02 12:42:55.486987,log_return_and_spread_features,1.000329,0.00200,3,"[0, 1]",2


In [ ]:
df_res.sort_values(by=["rmse"], ascending=True).tail(20)

,model_name,mse,rmse,mae,rmspe,mape,added_date,feature,time_taken,learning_rate,epochs,stock_ids,stock_ids_count
16,TCN,0.019632,0.140113,0.109476,NaN,NaN,2023-10-02 12:44:33.951629,log_return_spread_and_statistics_features,26.917324,0.01000,3,"[0, 1]",2
25,TCN,0.020974,0.144822,0.114959,NaN,NaN,2023-10-02 12:47:07.566162,volume_nn_features,26.365644,0.01000,3,"[0, 1]",2
30,TCN,0.021009,0.144944,0.100952,NaN,NaN,2023-10-02 12:48:23.836452,size_nn_features,26.453824,0.01000,3,"[0, 1]",2
17,LSTM,0.023194,0.152296,0.125939,NaN,NaN,2023-10-02 12:45:03.926532,log_return_spread_and_statistics_features,29.962694,0.01000,3,"[0, 1]",2
19,CNN,0.023832,0.154377,0.147918,50.817989,44.732070,2023-10-02 12:45:24.095427,price_nn_features,3.523589,0.00038,3,"[0, 1]",2
20,TCN,0.024041,0.155053,0.129801,NaN,NaN,2023-10-02 12:45:50.975379,price_nn_features,26.862900,0.01000,3,"[0, 1]",2
11,TCN,0.024547,0.156676,0.129240,NaN,NaN,2023-10-02 12:43:24.554129,log_return_and_spread_features,25.851582,0.01000,3,"[0, 1]",2
35,TCN,0.024926,0.157881,0.130381,NaN,NaN,2023-10-02 12:49:47.231162,random_nn_features,26.652972,0.01000,3,"[0, 1]",2
1,CNN,0.026277,0.162101,0.161746,54.656174,49.872181,2023-10-02 12:40:42.487133,rv_lags,3.613407,0.00038,3,"[0, 1]",2
21,LSTM,0.026878,0.163945,0.116280,NaN,NaN,2023-10-02 12:46:23.004723,price_nn_features,32.017688,0.01000,3,"[0, 1]",2
